In [ ]:

from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [ ]:
## Used the inbuilt tool of wikipedia
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
tools=[wiki,arxiv]

In [ ]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import openai
load_dotenv()
import os

groq_api_key=os.getenv("GROQ_API_KEY")
openai.api_key=os.getenv("OPENAI_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-it")


In [ ]:
## Custom tools[RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith-search","Search any information about Langsmith ")

retriever_tool.name
tools=[wiki,arxiv,retriever_tool]

In [ ]:
## Prompt Template
from langchain import hub
prompt=hub.pull("hwchase17/openai-functions-agent")


In [ ]:
## Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)
agent

In [35]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks witMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without being explicitly programmed.  


> Finished chain.


{'input': 'What is machine learning',
 'output': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without being explicitly programmed.  \n'}